In [2]:
import fastbook
fastbook.setup_book()

In [4]:
from fastai.vision.all import *
from fastbook import *

In [5]:
# Access MNIST dataset from fastai
path = untar_data(URLs.MNIST_SAMPLE)

In [6]:
Path.BASE_PATH = path

In [8]:
# See the directory contents for the MNIST dataset
path.ls()

(#3) [Path('train'),Path('valid'),Path('labels.csv')]

In [9]:
# See the contents of our train folder - '3' and '7' will be our labels
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

In [10]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [12]:
# Use Image class from Python Imaging Library (PIL) to open a sample image of 3
im_3_path = threes[0]
im_3 = Image.open(im_3_path)
im_3

To see the numbers that make up this image - we need to cover it to either a *NumPy Array* or a *PyTorch Tensor*.

The `4:10` in the code below represents that we request the rows/columns from index 4 (**inclusive**) to index 10 (**not inclusive**). So this is basically showing the top-left corner of the image since the indices work from top-to-bottom and left-to-right.

In [ ]:
# NumPy Array
array(im_3)[]